In [1]:
#checking tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#importing libraries 

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
# Cross Validation dataset links

#CV-1
train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 1/Training/"
val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 1/Validation/"

#CV-2
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 2/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 2/Validation/"

#CV-3
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 3/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 3/Validation/"

#CV-4
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 4/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 4/Validation/"

#CV-5
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 5/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 5/Validation/"

In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 31206 files belonging to 5 classes.


2023-01-05 16:33:33.212525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 16:33:33.248948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 16:33:33.249393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 16:33:33.284726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 7787 files belonging to 5 classes.


In [5]:
#Normalizing the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
#import tensorflow.python.keras.applications.VGG16
base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [14]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [15]:
#defining callback 
class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [16]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-01-05 16:33:37.563569: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-05 16:33:37.795100: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


244/244 [==============================] - ETA: 0s - loss: 1.1813 - accuracy: 0.5083 - sensitivity_at_specificity: 0.8949 - specificity_at_sensitivity: 0.8901 - recall: 0.2939 - precision: 0.7152
Epoch 1: val_accuracy improved from -inf to 0.60408, saving model to ECG_Spectrogram_Model.h5
244/244 [==============================] - 205s 792ms/step - loss: 1.1813 - accuracy: 0.5083 - sensitivity_at_specificity: 0.8949 - specificity_at_sensitivity: 0.8901 - recall: 0.2939 - precision: 0.7152 - val_loss: 0.9983 - val_accuracy: 0.6041 - val_sensitivity_at_specificity: 0.9512 - val_specificity_at_sensitivity: 0.9384 - val_recall: 0.4125 - val_precision: 0.7322
Epoch 2/50
244/244 [==============================] - ETA: 0s - loss: 0.8632 - accuracy: 0.6559 - sensitivity_at_specificity: 0.9772 - specificity_at_sensitivity: 0.9637 - recall: 0.5382 - precision: 0.7529
Epoch 2: val_accuracy improved from 0.60408 to 0.61667, saving model to ECG_Spectrogram_Model.h5
244/244 [========================

Epoch 14/50
244/244 [==============================] - ETA: 0s - loss: 0.5435 - accuracy: 0.7955 - sensitivity_at_specificity: 0.9965 - specificity_at_sensitivity: 0.9937 - recall: 0.7500 - precision: 0.8350
Epoch 14: val_accuracy did not improve from 0.71183
244/244 [==============================] - 183s 748ms/step - loss: 0.5435 - accuracy: 0.7955 - sensitivity_at_specificity: 0.9965 - specificity_at_sensitivity: 0.9937 - recall: 0.7500 - precision: 0.8350 - val_loss: 0.8551 - val_accuracy: 0.6921 - val_sensitivity_at_specificity: 0.9739 - val_specificity_at_sensitivity: 0.9723 - val_recall: 0.6412 - val_precision: 0.7314
Epoch 15/50
244/244 [==============================] - ETA: 0s - loss: 0.5325 - accuracy: 0.7994 - sensitivity_at_specificity: 0.9962 - specificity_at_sensitivity: 0.9938 - recall: 0.7563 - precision: 0.8389
Epoch 15: val_accuracy improved from 0.71183 to 0.72005, saving model to ECG_Spectrogram_Model.h5
244/244 [==============================] - 183s 750ms/step - 

Epoch 27/50
244/244 [==============================] - ETA: 0s - loss: 0.4278 - accuracy: 0.8394 - sensitivity_at_specificity: 0.9985 - specificity_at_sensitivity: 0.9970 - recall: 0.8115 - precision: 0.8663
Epoch 27: val_accuracy did not improve from 0.72005
244/244 [==============================] - 177s 725ms/step - loss: 0.4278 - accuracy: 0.8394 - sensitivity_at_specificity: 0.9985 - specificity_at_sensitivity: 0.9970 - recall: 0.8115 - precision: 0.8663 - val_loss: 1.1532 - val_accuracy: 0.6752 - val_sensitivity_at_specificity: 0.9337 - val_specificity_at_sensitivity: 0.9674 - val_recall: 0.6502 - val_precision: 0.7012
Epoch 28/50
244/244 [==============================] - ETA: 0s - loss: 0.4288 - accuracy: 0.8390 - sensitivity_at_specificity: 0.9986 - specificity_at_sensitivity: 0.9973 - recall: 0.8105 - precision: 0.8661
Epoch 28: val_accuracy did not improve from 0.72005
244/244 [==============================] - 177s 725ms/step - loss: 0.4288 - accuracy: 0.8390 - sensitivity_

Epoch 40/50
244/244 [==============================] - ETA: 0s - loss: 0.3287 - accuracy: 0.8775 - sensitivity_at_specificity: 0.9985 - specificity_at_sensitivity: 0.9991 - recall: 0.8596 - precision: 0.8960
Epoch 40: val_accuracy did not improve from 0.72005
244/244 [==============================] - 177s 725ms/step - loss: 0.3287 - accuracy: 0.8775 - sensitivity_at_specificity: 0.9985 - specificity_at_sensitivity: 0.9991 - recall: 0.8596 - precision: 0.8960 - val_loss: 1.3622 - val_accuracy: 0.6847 - val_sensitivity_at_specificity: 0.9253 - val_specificity_at_sensitivity: 0.9652 - val_recall: 0.6614 - val_precision: 0.7019
Epoch 41/50
244/244 [==============================] - ETA: 0s - loss: 0.3320 - accuracy: 0.8762 - sensitivity_at_specificity: 0.9988 - specificity_at_sensitivity: 0.9990 - recall: 0.8574 - precision: 0.8939
Epoch 41: val_accuracy did not improve from 0.72005
244/244 [==============================] - 177s 726ms/step - loss: 0.3320 - accuracy: 0.8762 - sensitivity_

In [17]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8954367637634277
Validation Accuracy:  0.7200462222099304
Validation Specificity:  0.9774303436279297
Validation Sensitivity:  0.97611403465271
Validation Recall:  0.6840888857841492
Validation Precision:  0.758441150188446
Validation Loss:  0.8061150312423706
